https://github.com/pranavphoenix/BiLSTM-POS-Tagging/blob/main/BiLSTM_POS_Tagging.ipynb

TODO:

- evitare di utilizzare la punctuation nel calcolo delle metriche, magari utilizzando l'array di pesi 'sample_weight' che si trova nell'altro notebook;

- Provare se i risultati migliorano con preprocessing (e.g. lowerando le parole)+ preprocessing di termini strani;
- Aggiustare il notebook perché fa cagare;

In [2]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import seaborn as sns
from collections import defaultdict
from keras import backend as K
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed
from keras.layers import Embedding, Activation
from keras.models import Sequential
from keras.optimizers import Adam
from keras_preprocessing.sequence import pad_sequences


import urllib.request
import zipfile
import progressbar

In [3]:
#Downloading the dataset
nltk.download('treebank')

data = nltk.corpus.treebank.tagged_sents()

#Downloading Glove Word Embeddings
pbar = None
def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None

# Download the GloVe embeddings file
url = 'http://nlp.stanford.edu/data/glove.6B.zip'
urllib.request.urlretrieve(url, 'glove.6B.zip', show_progress)

# Extract the zip file
zip_ref = zipfile.ZipFile('glove.6B.zip', 'r')
zip_ref.extractall()
zip_ref.close()

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
100% (862182613 of 862182613) |##########| Elapsed Time: 0:02:41 Time:  0:02:41


In [4]:
#Function to ignore the 0 padding while calculating accuracy
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

#Function to return one code encoding of tags
def one_hot_encoding(tag_sents, n_tags):
    tag_one_hot_sent = []
    for tag_sent in tag_sents:
        tags_one_hot = []
        for tag in tag_sent:
            tags_one_hot.append(np.zeros(n_tags))
            tags_one_hot[-1][tag] = 1.0
        tag_one_hot_sent.append(tags_one_hot)
    return np.array(tag_one_hot_sent)

#Function to convert output into tags
def logits_to_tags(tag_sentences, index):
    tag_sequences = []
    for tag_sentence in tag_sentences:
        tag_sequence = []
        for tag in tag_sentence:
            # if index[np.argmax(tag)] == "-PAD-":
            #     break
            # else:
                tag_sequence.append(index[np.argmax(tag)])
        tag_sequences.append(np.array(tag_sequence))
    return tag_sequences

In [5]:
# Get the files' list
fileids = nltk.corpus.treebank.fileids()

# Get the Penn Treebank corpus and tokenize the text
train_corpus = nltk.corpus.treebank.tagged_sents(fileids[:100])
val_corpus = nltk.corpus.treebank.tagged_sents(fileids[100:150])
test_corpus = nltk.corpus.treebank.tagged_sents(fileids[150:])

# Flatten the lists
#ignore = [':', '"', '-LRB-', '-RRB-', ',', '.', "''", '``', 'SYM']

# train_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(train_corpus) for item in sublist if item[1] != '-NONE-']
# val_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(val_corpus) for item in sublist if item[1] != '-NONE-']
# test_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(test_corpus) for item in sublist if item[1] != '-NONE-']

train_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(train_corpus) for item in sublist]
val_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(val_corpus) for item in sublist]
test_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(test_corpus) for item in sublist]

train_df = pd.DataFrame(train_corpus, columns = ['word', 'tag', 'sentence'])
val_df = pd.DataFrame(val_corpus, columns = ['word', 'tag', 'sentence'])
test_df = pd.DataFrame(test_corpus, columns = ['word', 'tag', 'sentence'])
print(train_df)

         word     tag sentence
0      Pierre     NNP        0
1      Vinken     NNP        0
2           ,       ,        0
3          61      CD        0
4       years     NNS        0
...       ...     ...      ...
50743      he     PRP     1962
50744     has     VBZ     1962
50745   faced     VBN     1962
50746   *T*-2  -NONE-     1962
50747       .       .     1962

[50748 rows x 3 columns]


In [6]:
tags_train = sorted(list(set([x for x in train_df.tag])))
tags_val = sorted(list(set([x for x in val_df.tag])))
tags_test = sorted(list(set([x for x in test_df.tag])))

print('Train tags number:',len(tags_train))
print('Val tags number:',len(tags_val))
print('Test tags number:',len(tags_test))


Train tags number: 46
Val tags number: 45
Test tags number: 41


---

###Vocabulary

GloVe Vocabulary (V1)

In [30]:
#Use the 300 dimensional GLove Word Embeddings
glove_dir = './'

embeddings_index =  {} #initialize dictionary
f = open(os.path.join(glove_dir, 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


V1 + Training set OOV (V2)

In [31]:
embedding_dim = 300

def update_vocab(df,embeddings_index,embedding_dim):
  oov_c = 0
  for word in df.word:
    if word not in embeddings_index:
      oov_c += 1
      random_embed = np.random.rand(embedding_dim)
      embeddings_index[word] = random_embed
  print("Added",oov_c,"OOV words + respective embeddings to the vocabulary.")
  return embeddings_index

embeddings_index = update_vocab(train_df,embeddings_index,embedding_dim)

Added 2777 OOV words + respective embeddings to the vocabulary.


V2 + Validation set OOV (V3)

In [32]:
embeddings_index = update_vocab(val_df,embeddings_index,embedding_dim)

Added 951 OOV words + respective embeddings to the vocabulary.


V3 + Test set OOV (V4)

In [33]:
embeddings_index = update_vocab(test_df,embeddings_index,embedding_dim)

Added 455 OOV words + respective embeddings to the vocabulary.


In [34]:
#Building the actual word vocabulary

from collections import OrderedDict

index2word = OrderedDict()
word2index = OrderedDict()
    
curr_idx = 0
for key in embeddings_index.keys():
  word2index[key] = curr_idx
  index2word[curr_idx] = key
  curr_idx += 1

#word_listing = list(index2word.values())

print(f'[Debug] Index -> Word vocabulary size: {len(index2word)}')
print(f'[Debug] Word -> Index vocabulary size: {len(word2index)}')

[Debug] Index -> Word vocabulary size: 404183
[Debug] Word -> Index vocabulary size: 404183


In [37]:
#Tag vocabulary

tag2index = OrderedDict()
index2tag = OrderedDict()

curr_id = 0

for tag in tags_train:
  tag2index[tag] = curr_id
  index2tag[curr_id] = tag
  curr_id += 1

print(f'[Debug] Index -> Tag vocabulary size: {len(index2tag)}')
print(f'[Debug] Tag -> Index vocabulary size: {len(tag2index)}')


[Debug] Index -> Tag vocabulary size: 46
[Debug] Tag -> Index vocabulary size: 46


---

In [38]:
acc = []
conf_matrix = []

tag_list = tags_train
# The integers for each tag are the same as above

MAX_LENGTH = max([len(seq) for seq in data]) # maximum words in a sentence

conf_mat_df = pd.DataFrame(columns=tag_list, index=tag_list)
conf_mat_df = conf_mat_df.fillna(0)

In [39]:
train_sentences = train_df.groupby('sentence').word.apply(list).reset_index()['word']
val_sentences = val_df.groupby('sentence').word.apply(list).reset_index()['word']
test_sentences = test_df.groupby('sentence').word.apply(list).reset_index()['word']

train_tags = train_df.groupby('sentence').tag.apply(list).reset_index()['tag']
val_tags = val_df.groupby('sentence').tag.apply(list).reset_index()['tag']
test_tags = test_df.groupby('sentence').tag.apply(list).reset_index()['tag']

In [40]:
#Tokenising words and  by their indexes in vocabulary
train_sentences_X, test_sentences_X, val_sentences_X, train_tags_y, test_tags_y, val_tags_y = [], [], [], [], [], []

for sentence in train_sentences:
    sent_int = []
    for word in sentence:
        sent_int.append(word2index[word])
   
    train_sentences_X.append(sent_int)

for sentence in val_sentences:
    sent_int = []
    for word in sentence:
        sent_int.append(word2index[word])

    val_sentences_X.append(sent_int)

for sentence in test_sentences:
    sent_int = []
    for word in sentence:
        sent_int.append(word2index[word])

    test_sentences_X.append(sent_int)

for sent_tags in train_tags:
    train_tags_y.append([tag2index[tag] for tag in sent_tags])

for sent_tags in val_tags:
    val_tags_y.append([tag2index[tag] for tag in sent_tags])

for sent_tags in test_tags:
    test_tags_y.append([tag2index[tag] for tag in sent_tags])

#Add padding to sentences
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
val_sentences_X = pad_sequences(val_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')

train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
val_tags_y = pad_sequences(val_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')



###ANCHE LA PARTE DELL'EMBEDDING LAYER E' ADATTATA AL NUOVO VOCABOLARIO 

In [46]:
#Building the Embedding Layer 
embedding_dim = 300

embedding_matrix = np.zeros((len(word2index), embedding_dim))
for word, i in word2index.items():
    embedding_vector = embeddings_index.get(word)
    if i < len(word2index):
        embedding_matrix[i] = embedding_vector

#Building the BiLSTM model
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, ))) 
model.add(Embedding(len(word2index), 300, weights=[embedding_matrix],trainable=False))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])
model.summary()
one_hot_train_tags_y = one_hot_encoding(train_tags_y, len(tag2index))


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 271, 300)          121254900 
                                                                 
 bidirectional_3 (Bidirectio  (None, 271, 512)         1140736   
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 271, 46)          23598     
 tributed)                                                       
                                                                 
 activation_3 (Activation)   (None, 271, 46)           0         
                                                                 
Total params: 122,419,234
Trainable params: 1,164,334
Non-trainable params: 121,254,900
_________________________________________________________________


In [47]:
history = model.fit(train_sentences_X, one_hot_encoding(train_tags_y, len(tag2index)),\
          validation_data=(val_sentences_X, one_hot_encoding(val_tags_y, len(tag2index))),
          batch_size=128, epochs= 15)


Epoch 1/15
16/16 [==============================] - 168s 11s/step - loss: 0.7329 - accuracy: 0.8505 - ignore_accuracy: 0.3705 - val_loss: 0.3099 - val_accuracy: 0.9186 - val_ignore_accuracy: 0.1618
Epoch 2/15
16/16 [==============================] - 106s 7s/step - loss: 0.2856 - accuracy: 0.9249 - ignore_accuracy: 0.2253 - val_loss: 0.2629 - val_accuracy: 0.9323 - val_ignore_accuracy: 0.2929
Epoch 3/15
16/16 [==============================] - 103s 6s/step - loss: 0.2462 - accuracy: 0.9363 - ignore_accuracy: 0.3473 - val_loss: 0.2341 - val_accuracy: 0.9403 - val_ignore_accuracy: 0.3715
Epoch 4/15
16/16 [==============================] - 103s 6s/step - loss: 0.2188 - accuracy: 0.9464 - ignore_accuracy: 0.4427 - val_loss: 0.2086 - val_accuracy: 0.9483 - val_ignore_accuracy: 0.4489
Epoch 5/15
16/16 [==============================] - 106s 7s/step - loss: 0.1925 - accuracy: 0.9525 - ignore_accuracy: 0.5051 - val_loss: 0.1822 - val_accuracy: 0.9560 - val_ignore_accuracy: 0.5299
Epoch 6/15
16/

In [48]:
scores = model.evaluate(test_sentences_X, one_hot_encoding(test_tags_y, len(tag2index)))
acc.append(scores[2]*100)


predictions = model.predict(test_sentences_X)


21/21 [==============================] - 16s 710ms/step


In [49]:
ignore = [':', '#', '$', '-LRB-', '-RRB-', ',', '.', "''", '``', 'SYM', '-NONE-']

# Convert the class probabilities into class labels
predicted_labels = np.argmax(predictions, axis=-1)

true_pos = defaultdict(int)
false_pos = defaultdict(int)
false_neg = defaultdict(int)
precision = defaultdict(float)
recall = defaultdict(float)
f1score = defaultdict(float)

#index2tag = {v: k for k, v in tag2index.items()}
for tag in tags_test:
  if tag not in ignore:
    for idx_sentence in range(len(test_tags_y)):
      for idx_word in range(MAX_LENGTH):
        if index2tag[test_tags_y[idx_sentence][idx_word]] == tag:
          # If the predicted tag matches ground truth we increase true positive count for that tag
          if predicted_labels[idx_sentence][idx_word] == test_tags_y[idx_sentence][idx_word]:
            true_pos[index2tag[test_tags_y[idx_sentence][idx_word]]] += 1
          else:
          # If the predicted tag does not match ground truth we increase false negative count for that tag
          # and false positive count for the wrongly predicted tag
            false_neg[index2tag[test_tags_y[idx_sentence][idx_word]]] += 1
            false_pos[index2tag[predicted_labels[idx_sentence][idx_word]]] += 1
    # We then compute precision, recall and F1 scores
    if true_pos[tag] != 0:
      precision[tag] = true_pos[tag] / (true_pos[tag] + false_pos[tag])
      recall[tag] = true_pos[tag] / (true_pos[tag] + false_neg[tag])
      f1score[tag] = 2 * ((precision[tag] * recall[tag]) / (precision[tag] + recall[tag])) 
    else:
      print(tag)
      print(true_pos[tag])
      print(false_pos[tag])
      print(false_neg[tag])
      print()

EX
0
0
5

JJR
0
0
59

JJS
0
0
31

NNPS
0
0
44

PDT
0
0
4

RBR
0
0
15

RBS
0
0
3

RP
0
0
33

WP$
0
0
4

WRB
0
0
24



In [50]:
for tag in tags_test:
  if tag not in ignore:
    print(f'Tag: {tag}')
    print(f'\tPrecision: {precision[tag]}')
    print(f'\tRecall: {recall[tag]}')
    print(f'\tF1-score: {f1score[tag]}')

Tag: CC
	Precision: 1.0
	Recall: 0.9344262295081968
	F1-score: 0.9661016949152543
Tag: CD
	Precision: 1.0
	Recall: 0.824009324009324
	F1-score: 0.9035143769968051
Tag: DT
	Precision: 0.9868319132455461
	Recall: 0.954307116104869
	F1-score: 0.9702970297029703
Tag: EX
	Precision: 0.0
	Recall: 0.0
	F1-score: 0.0
Tag: IN
	Precision: 0.9900727994705493
	Recall: 0.9177914110429448
	F1-score: 0.9525628780643106
Tag: JJ
	Precision: 0.9425531914893617
	Recall: 0.48257080610021785
	F1-score: 0.638328530259366
Tag: JJR
	Precision: 0.0
	Recall: 0.0
	F1-score: 0.0
Tag: JJS
	Precision: 0.0
	Recall: 0.0
	F1-score: 0.0
Tag: MD
	Precision: 1.0
	Recall: 0.9520958083832335
	F1-score: 0.9754601226993864
Tag: NN
	Precision: 0.8698224852071006
	Recall: 0.863617289131347
	F1-score: 0.8667087807959569
Tag: NNP
	Precision: 0.861904761904762
	Recall: 0.9627659574468085
	F1-score: 0.9095477386934673
Tag: NNPS
	Precision: 0.0
	Recall: 0.0
	F1-score: 0.0
Tag: NNS
	Precision: 0.8155216284987278
	Recall: 0.681190223

In [51]:
provola = [el[1] for el in f1score.items()]

sum = 0

for p in provola:
  sum += p

print(f'MACRO F1-score: {sum / len(f1score.items())}')

MACRO F1-score: 0.5824268549656594
